# **PR: weight matrix**

Dong Gyun Ko <br/>
last Updated: july 20, 2022

In [27]:
import numpy as np
import pandas as pd

## **1. import data**

In [28]:
url1 = 'https://raw.githubusercontent.com/QuantEcon/lecture-python/master/source/_static/lecture_specific/pandas_panel/realwage.csv'

In [29]:
# display 6 columns for viewing purposes
pd.set_option('display.max_columns', 6)

# reduce decimal points to 2
pd.options.display.float_format = '{:,.2f}'.format

# read url1
df_url = pd.read_csv(url1)

# condition
cond1 = (df_url['Country'] == 'United States')
cond2 = (df_url['Country'] == 'United Kingdom')
cond3 = (df_url['Country'] == 'Ireland')
cond4 = (df_url['Country'] == 'Japan')

df_url = df_url.loc[cond1|cond2|cond3|cond4]

# pivot_table
df_url = df_url.pivot_table(values='value',
                            index='Time',
                            columns=['Country', 'Series', 'Pay period'])

# to_datetime
df_url.index = pd.to_datetime(df_url.index)

# xs
df_url_f = df_url.xs(('Hourly', 'In 2015 constant prices at 2015 USD exchange rates'), 
                     level=('Pay period', 'Series'), axis=1)

# stack
df = pd.DataFrame(df_url_f.stack(level='Country')).rename(columns={0:'value'})

# reset the index
df.reset_index(inplace=True)

# generate sum & ratio
df['sum'] = df.groupby(['Time'])['value'].transform('sum')
df['ratio'] = df['value']/df['sum']

In [30]:
# sample data(t0: 2006, T:11, N:4)

df

,Time,Country,value,sum,ratio
0,2006-01-01,Ireland,9.18,30.73,0.30
1,2006-01-01,Japan,5.69,30.73,0.19
2,2006-01-01,United Kingdom,9.81,30.73,0.32
3,2006-01-01,United States,6.05,30.73,0.20
4,2007-01-01,Ireland,9.70,31.76,0.31
5,2007-01-01,Japan,5.75,31.76,0.18
6,2007-01-01,United Kingdom,10.07,31.76,0.32
7,2007-01-01,United States,6.24,31.76,0.20
8,2008-01-01,Ireland,9.51,32.13,0.30
9,2008-01-01,Japan,5.79,32.13,0.18


## **2. weight matrix**

In [31]:
# weight_matrix(init_t, num_t, num_n)

def weight_matrix(t0, T, N):

    for i in range(0,T):

        globals()['df_{}'.format(i+t0)] = df.loc[N*i:N*i+(N-1)]
    
    for j in range(t0,t0+T):

        row_0 = np.array(globals()['df_{}'.format(j)]['ratio'].iloc[0:len(globals()['df_{}'.format(j)])]).reshape(1,-1)

        for k in range(0,len(globals()['df_{}'.format(j)])):

            globals()['row_{}'.format(k+1)] = np.roll(row_0, k)
    
        globals()['weight_matrix_{}'.format(j)] = np.asmatrix(np.vstack([globals()['row_{}'.format(k+1)] for k in range (0,len(globals()['df_{}'.format(j)]))]))

    zero_vec = np.hstack([0] for v in range(0,N))

    for j in range(t0,t0+T):

        np.fill_diagonal(globals()['weight_matrix_{}'.format(j)], val = zero_vec)
    
    for j in range(t0,t0+T):

        print('{} year weight matrix'.format(j))
        print(globals()['weight_matrix_{}'.format(j)])

In [32]:
weight_matrix(2006, 11, 4)

2006 year weight matrix
[[0.         0.18519362 0.31910185 0.19703872]
 [0.19703872 0.         0.18519362 0.31910185]
 [0.31910185 0.19703872 0.         0.18519362]
 [0.18519362 0.31910185 0.19703872 0.        ]]
2007 year weight matrix
[[0.         0.18103096 0.31709545 0.19652361]
 [0.19652361 0.         0.18103096 0.31709545]
 [0.31709545 0.19652361 0.         0.18103096]
 [0.18103096 0.31709545 0.19652361 0.        ]]
2008 year weight matrix
[[0.         0.1802901  0.31267509 0.21107514]
 [0.21107514 0.         0.1802901  0.31267509]
 [0.31267509 0.21107514 0.         0.1802901 ]
 [0.1802901  0.31267509 0.21107514 0.        ]]
2009 year weight matrix
[[0.         0.17801995 0.3013422  0.22505642]
 [0.22505642 0.         0.17801995 0.3013422 ]
 [0.3013422  0.22505642 0.         0.17801995]
 [0.17801995 0.3013422  0.22505642 0.        ]]
2010 year weight matrix
[[0.         0.18041555 0.29270873 0.23158081]
 [0.23158081 0.         0.18041555 0.29270873]
 [0.29270873 0.23158081 0.    

C:\Users\geodo\AppData\Local\Temp\ipykernel_50016\421054174.py:19: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  zero_vec = np.hstack([0] for v in range(0,N))
